# Demo: Run an esmvaltool recipe using containers

In this example, we demonstrate how to execute an ESMValtool recipe using containers on Snellius infrastructure.

## Prerequest: Starting a Jupyter Server on Snellius.

A Jupyter Server should be started on Snellius as the environment to execute this notebook. You can follow [this instruction](https://github.com/RS-DAT/JupyterDaskOnSLURM) to start a Jupyter server on Senllius.

After successfully setting up the Jupyter server, please copy this notebook to the Snellius file system. Then open it from the browser on your local PC (as specified in the above instructon). Effectively, this notebook run as a Slurm job on Snellius.

## Step 1: build a Singularity container

ESMValTool is provided in the form of Docker containers. However on Snellius, [it is suggested to use Singularity](https://servicedesk.surf.nl/wiki/pages/viewpage.action?pageId=30660251) as the container technology. Therefore we will convert the existing Docker container to a Singularity container. Please run this command on Snellius:

```sh
# Step1: build sif image (this should be done once)
singularity build esmvaltool_stable.sif docker://esmvalgroup/esmvaltool:stable
```

This will build a Singularity Image File (.sif) named `esmvaltool_stable.sif` on the Snellius file system, from DockerHub. Note that this may take ~20 minutes, and can only be executed on the `cbuild` partition on Snellius.

## Step 2: User configeration
One can run the following command to get the user configeration file.

In [ ]:
# Get user config file
!singularity run esmvaltool_stable.sif config get_config_user

## Step 3: Execute receipy

One can run the following command to run two recipes parallely: 'recipe_jan.yml', 'recipe_feb.yml'

In [1]:
from dask.distributed import Client

client = Client("tcp://145.136.56.217:33227")
client

<Client: 'tcp://145.136.56.217:33227' processes=3 threads=96, memory=90.00 GiB>

-- Add here the cluster --

In [2]:
recipies = ['recipe_jan.yml', 'recipe_feb.yml']
commands = ['singularity run esmvaltool_stable.sif run recipes/{} --offline=False'.format(r) for r in recipies]

commands

['singularity run esmvaltool_stable.sif run recipes/recipe_jan.yml --offline=False',
 'singularity run esmvaltool_stable.sif run recipes/recipe_feb.yml --offline=False']

In [3]:
import dask, os

run_func = os.system

futures = client.compute( [run_func(c) for c in commands])

2022-09-25 19:09:31,091 UTC [1173935] INFO    
______________________________________________________________________
          _____ ____  __  ____     __    _ _____           _
         | ____/ ___||  \/  \ \   / /_ _| |_   _|__   ___ | |
         |  _| \___ \| |\/| |\ \ / / _` | | | |/ _ \ / _ \| |
         | |___ ___) | |  | | \ V / (_| | | | | (_) | (_) | |
         |_____|____/|_|  |_|  \_/ \__,_|_| |_|\___/ \___/|_|
______________________________________________________________________

ESMValTool - Earth System Model Evaluation Tool.

http://www.esmvaltool.org

CORE DEVELOPMENT TEAM AND CONTACTS:
  Birgit Hassler (Co-PI; DLR, Germany - birgit.hassler@dlr.de)
  Alistair Sellar (Co-PI; Met Office, UK - alistair.sellar@metoffice.gov.uk)
  Bouwe Andela (Netherlands eScience Center, The Netherlands - b.andela@esciencecenter.nl)
  Lee de Mora (PML, UK - ledm@pml.ac.uk)
  Niels Drost (Netherlands eScience Center, The Netherlands - n.drost@esciencecenter.nl)
  Veronika Eyring (DLR, Ger

Once finished, one can check the downloaded climate data files and the generated results:

In [4]:
# Check generated results
!tree ~/esmvaltool_output/

/home/ouku1/esmvaltool_output/
├── recipe_feb_20220925_190954
│   ├── index.html
│   ├── plots
│   │   ├── map
│   │   │   └── script1
│   │   │       └── png
│   │   │           ├── CMIP5_CanESM2_Amon_historical_r1i1p1_tas_2000-2000.png
│   │   │           ├── CMIP5_CanESM2_Amon_historical_r1i1p1_tas_2000-2000_citation.bibtex
│   │   │           ├── CMIP5_CanESM2_Amon_historical_r1i1p1_tas_2000-2000_data_citation_info.txt
│   │   │           ├── CMIP5_CanESM2_Amon_historical_r1i1p1_tas_2000-2000_provenance.xml
│   │   │           ├── CMIP6_BCC-ESM1_Amon_historical_r1i1p1f1_tas_gn_2000-2000.png
│   │   │           ├── CMIP6_BCC-ESM1_Amon_historical_r1i1p1f1_tas_gn_2000-2000_citation.bibtex
│   │   │           ├── CMIP6_BCC-ESM1_Amon_historical_r1i1p1f1_tas_gn_2000-2000_data_citation_info.txt
│   │   │           └── CMIP6_BCC-ESM1_Amon_historical_r1i1p1f1_tas_gn_2000-2000_provenance.xml
│   │   └── timeseries
│   │       └── script1
│   │           └── png
│   │               ├── tas_am